In [168]:
import pandas as pd
import numpy as np
import re
import os

In [169]:
#Create a read file function that read in all csv, and xlsx files in a folder. For each csv create a new DataFrame, For each xlsx has multiple sheets, create new DataFrame for each sheets. Then append new Data Frame to DF_List.

#based code from Nielsen loop.ipynb

directory = r'C:\Users\GLai\OneDrive - California Department of Toxic Substances Control\Documents\SCP marketecon research\PFAS Treatment priority product regulation\Product Data Files'
df_list = []
#loop through xlsx in directory_path
for filename in os.listdir(directory):
    full_directory = os.path.join(directory, filename)
    print("now working on" + filename)

    if filename.endswith('.csv'):
        df = pd.read_csv(full_directory)
        df_list.append(df)

    elif filename.endswith('.xlsx'):
        xls = pd.ExcelFile(full_directory)
        sheets = xls.sheet_names

        if len(sheets) == 1:
            df = pd.read_excel(full_directory)

        else:
            for sheet_name in sheets:
                df = pd.read_excel(full_directory, sheet_name= sheet_name)
                df_list.append(df)
    print("finished working on" + filename)

now working on111.xlsx
finished working on111.xlsx
now working onCopy of scrape.xlsx
finished working onCopy of scrape.xlsx
now working onGRIFFIN.csv
finished working onGRIFFIN.csv
now working ongriotsgarage.csv
finished working ongriotsgarage.csv
now working onjason-markk.csv
finished working onjason-markk.csv
now working onLysol.csv
finished working onLysol.csv
now working onMalcoProducts.xlsx
finished working onMalcoProducts.xlsx
now working onmanual work.csv
finished working onmanual work.csv
now working onMasterBlend.csv
finished working onMasterBlend.csv
now working onmoneysworthandbest.csv
finished working onmoneysworthandbest.csv
now working onRepelWell (1).csv
finished working onRepelWell (1).csv
now working onresolve.csv
finished working onresolve.csv
now working onvanishoxi.csv
finished working onvanishoxi.csv
now working onwoolite.csv
finished working onwoolite.csv


In [170]:
#formating punctualtions and replace space with "_"
def clean_header(header):
    header = header.lower()
    header = re.sub(r'[^a-zA-Z0-9_]', ' ', header)
    header = header.replace(' ', '_')
    return header

In [171]:
#apply clean_header to dataframe headers
def clean_dataframe_header(df):
    df.columns = [clean_header(header) for header in df.columns]
    return df

In [172]:
#call clean_dataframe_header need to format because able to concat
[clean_dataframe_header(df) for df in df_list]

[   web_scraper_order                              web_scraper_start_url  page  \
 0       1691079787-1  https://reshoevn8r.com/collections/shoe-care-p...   NaN   
 1       1691079791-2  https://reshoevn8r.com/collections/shoe-care-p...   NaN   
 2       1691079806-3  https://reshoevn8r.com/collections/shoe-care-p...   NaN   
 3       1691079812-4  https://reshoevn8r.com/collections/shoe-care-p...   NaN   
 4       1691079816-5  https://reshoevn8r.com/collections/shoe-care-p...   NaN   
 5       1691079829-6  https://reshoevn8r.com/collections/shoe-care-p...   NaN   
 6       1691079842-7  https://reshoevn8r.com/collections/shoe-care-p...   NaN   
 7       1691079846-8  https://reshoevn8r.com/collections/shoe-care-p...   NaN   
 8       1691079850-9  https://reshoevn8r.com/collections/shoe-care-p...   NaN   
 9      1691079854-10  https://reshoevn8r.com/collections/shoe-care-p...   NaN   
 10     1691079858-11  https://reshoevn8r.com/collections/shoe-care-p...   NaN   
 11     16910798

In [173]:
# create new column with combining product name, column containing "description"
for df in df_list:
    description_columns = [col for col in df.columns if "description" in col]
    df.fillna('', inplace=True)
    df['filter'] = df['product_name'] + ' ' + df[description_columns].astype(str).apply(lambda x: ' '.join(x), axis=1)

In [174]:
for df in df_list:
        df.rename(columns= {'link_herf': 'link_href'})
        df.rename(columns= {'product_href': 'link_href'})


In [175]:
#concat all dataframes to single df
finaldf = pd.concat(df_list, ignore_index=True)

In [176]:
#format price column
finaldf['price'] = finaldf['price'].str.extract(r'\$(\d+\.\d+)') if not None else ' '

In [177]:
#extract company name through www addresses
def extract_company(link_href):
    if pd.isna(link_href):
        return ' '
    
    if 'www.' in link_href:
        start = link_href.find('www.') + len('www.')
    elif 'https://' in link_href:
        start = link_href.find('https://') + len('https://')
    elif 'http://' in link_href:
        start = link_href.find('http://') + len('http://')
    
    else:
        start = 0

    end = link_href.find('.com', start)
    if end == -1:
        return ' '
    
    return link_href[start:end]

finaldf['company'] = finaldf['link_href'].apply(extract_company)

In [178]:
#format filter row value to lower case and no punctuation
def clean_filter_column(filter_str):
    if not isinstance(filter_str, str):
        filter_str = str(filter_str)
    cleaned_str = re.sub(r'[^\w\s]', '', filter_str)
    cleaned_str = cleaned_str.lower()
    return cleaned_str

finaldf['filter'] = finaldf['filter'].apply(clean_filter_column)

In [179]:
# Keyword list
keywords_search = ['textile', 'carpets', 'carpet', 'upholstery', 'funiture', 'leather', 'rugs', 'rug', 'clothing', 'cloth', 'shoe', 'shoes', 'felt', 'suede', 'fabric', 'nylon']

keywords = ['cleaner', 'spot remover', 'remover', 
            'protectant', 'repellent', 'carpet', 
            'rug', 'clothing', 'shoe', 'upholstery', 
            'textile', 'leather', 'aftermarket', 
            'treatments', 'impregnating agents', 
            'proofer', 'furniture', 'eleminating', 
            'cleaner', 'protect', 'surface', 'absorb', 
            'absorbtion','protectant', 'spot remover', 
            'clean', 'remover', 'stains', 'water repellant', 
            'repellant', 'proofer', 'repellent']

keywords_without_products = ['cleaner', 'spot remover', 'remover', 
            'protectant', 'repellent', 'aftermarket', 
            'treatments', 'impregnating agents', 
            'proofer', 'eleminating', 
            'cleaner', 'protect', 'surface', 'absorb', 
            'absorbtion','protectant', 'spot remover', 
            'clean', 'remover', 'stains', 'water repellant', 
            'repellant', 'proofer', 'repellent', 'detergent']

In [180]:
#extract keyword and create new row with keyword containing list
def find_keywords(row):
    appeared_keywords = [keyword for keyword in keywords_without_products if keyword in row['filter']]
    return appeared_keywords

filtered_df = finaldf[finaldf['filter'].str.contains('|'.join(keywords_search))].copy()

filtered_df['appeared_keywords'] = filtered_df.apply(find_keywords,axis=1)


In [181]:
#product_category_dict
category_dict = {"Cleaner": ["eliminating", "cleaner", 'cleaning', 'clean'],
                 "Protectant": ["surface", "absorption", "protectant"],
                 "Spot remover": ["remover", "remove", "spot", "spots", "stain", "stains"],
                 "Water repellant": ["water", 'repel', 'repellant', 'proofer', 'repellent']
                 }

In [182]:
#match category product based on key_dict
def map_catgeory(row):
    categories = [category for category, keywords in category_dict.items() if any(keyword in row['appeared_keywords'] for keyword in keywords)]
    return categories if categories else [None]

filtered_df['product_category'] = filtered_df.apply(map_catgeory, axis=1)

In [183]:
filtered_dataframes_list= filtered_df[filtered_df['product_category'].apply(lambda x: x != [None])]

In [184]:
unique_combination_length = filtered_dataframes_list.drop_duplicates(subset= ['product_name'])

In [185]:
Outputdf = unique_combination_length[['company', 'product_name', 'link_href', 'product_category', 'price']]

In [186]:
#export cleaned df to file
Outputdf.to_csv('first.csv', index=False)